In [32]:
import os
os.chdir(os.path.abspath(os.curdir).replace('notebooks',''))

In [4]:
import json
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModel
import random
import pandas as pd
import boto3

In [6]:
from source.utils import flatten_list, create_path, get_keys

In [ ]:
list_rels = list(js_ls)

In [110]:
rel = list_rels[0]

half_way = int(len(js_ls[rel]) / 2)
half_way

350

In [116]:
random.sample(range(10),k=4)

[9, 4, 5, 6]

In [121]:
len(all_sents_ex_current)

44100

In [8]:
%%time



CPU times: user 886 µs, sys: 2.13 ms, total: 3.01 ms
Wall time: 10.8 s


0

In [19]:
method = 'bracket'
method = 'ignore'

model_path = f'./trained_models/distilbert-base-uncased-mtb-rnd_fel_rel_{method}/'
df_pred = pd.read_csv(f'{model_path}/test_results_None.txt',sep='\t',index_col=0)
df_test = pd.read_csv(f'./data/train_samples/few_rel_val_{method}_test.csv')

pd.DataFrame(classification_report(df_test.label.values,df_pred.prediction.values, output_dict=True)).round(2)

,0,1,accuracy,macro avg,weighted avg
precision,0.80,0.77,0.78,0.78,0.78
recall,0.76,0.81,0.78,0.78,0.78
f1-score,0.78,0.79,0.78,0.78,0.78
support,5600.00,5600.00,0.78,11200.00,11200.00


In [40]:

#method = 'ignore'



def get_results(method = 'bracket', model_class = 'distilbert-base-uncased',
     train_dataset = 'few_rel', eval_dataset = 'few_rel'):
    model_path = f'./trained_models/{model_class}-mtb-rnd_{train_dataset}_{method}'
    df_pred = pd.read_csv(f'{model_path}/test_results_None.txt',sep='\t',index_col=0)
    df_test = pd.read_csv(f'./data/train_samples/{eval_dataset}_val_{method}_test.csv')
    cls_rpt = classification_report(df_test.label.values,df_pred.prediction.values, output_dict=True)
    return pd.DataFrame(cls_rpt).round(2)

get_results(method = 'bracket', train_dataset = 'fel_rel', eval_dataset = 'few_rel')

,0,1,accuracy,macro avg,weighted avg
precision,0.84,0.86,0.85,0.85,0.85
recall,0.86,0.83,0.85,0.85,0.85
f1-score,0.85,0.84,0.85,0.85,0.85
support,5600.00,5600.00,0.85,11200.00,11200.00


In [42]:
get_results(method = 'mask_one', train_dataset = 'fel_rel', eval_dataset = 'few_rel')

,0,1,accuracy,macro avg,weighted avg
precision,0.79,0.77,0.78,0.78,0.78
recall,0.76,0.80,0.78,0.78,0.78
f1-score,0.77,0.78,0.78,0.78,0.78
support,5600.00,5600.00,0.78,11200.00,11200.00


In [43]:
get_results(method = 'ignore', train_dataset = 'fel_rel', eval_dataset = 'few_rel')

,0,1,accuracy,macro avg,weighted avg
precision,0.80,0.77,0.78,0.78,0.78
recall,0.76,0.81,0.78,0.78,0.78
f1-score,0.78,0.79,0.78,0.78,0.78
support,5600.00,5600.00,0.78,11200.00,11200.00


In [28]:
pd.read_csv('./data/SemEval2010_task8_all_data/SemEval2010_task8_testing_keys/TEST_FILE_CLEAN.TXT',
            sep='\t',header=None)

,0,1
0,8001,The most common <e1>audits</e1> were about <e2...
1,8002,The <e1>company</e1> fabricates plastic <e2>ch...
2,8003,The school <e1>master</e1> teaches the lesson ...
3,8004,The suspect dumped the dead <e1>body</e1> into...
4,8005,Avian <e1>influenza</e1> is an infectious dise...
...,...,...
2712,10713,"After seating all the idols, which itself take..."
2713,10714,The minister attributed the slow production of...
2714,10715,The <e1>umbrella</e1> <e2>frame</e2> is provid...
2715,10716,Manos: The Hands of Fate is a low-budget horro...


,0,1,accuracy,macro avg,weighted avg
precision,0.84,0.86,0.85,0.85,0.85
recall,0.86,0.83,0.85,0.85,0.85
f1-score,0.85,0.84,0.85,0.85,0.85
support,5600.00,5600.00,0.85,11200.00,11200.00


In [7]:
from sklearn.metrics import classification_report

In [19]:
round(246915368/10e5,1)

246.9

In [137]:

create_path(save_path)

In [138]:
random_state = 1701



In [172]:
model_name = 'distilbert-base-uncased-mtb-rnd'
exp = 'fel_rel_bracket'
create_path('./trained_models/')
p = dict(
    model_path = f'./models/{model_name}/',
    max_len = 128*2,
    tr_ep = 15,
    output_dir = f'./trained_models/{model_name}_{exp}/',
    file = './data/train_samples/few_rel_train_bracket',
    test_mode = True,
    nr_test = 10
)
create_run_glue_script(p)

./trained_models/ exists already
python run_glue.py        --model_name_or_path ./models/distilbert-base-uncased-mtb-rnd/        --do_train        --do_eval        --do_predict        --max_seq_length 256        --num_train_samples 15        --output_dir ./trained_models/distilbert-base-uncased-mtb-rnd_fel_rel_bracket/        --train_file ./data/train_samples/few_rel_train_bracket_train.csv        --validation_file ./data/train_samples/few_rel_train_bracket_val.csv        --test_file ./data/train_samples/few_rel_train_bracket_test.csv --max_train_samples 10        --max_val_samples 10        --max_test_samples 10


In [165]:
python source/add_entity_tokens.py --model_name distilbert-base-uncased
python get_data.py
python create_train_data.py
python 


RangeIndex(start=40320, stop=44800, step=1)

In [152]:
test_len

4480

In [ ]:
main driver

x driven by ...
x caused by ...

examples of x is

In [ ]:
import requests

file_names = ['val_wiki','train_wiki']
is not os.path.exists('./data'):
    os.mkdir('./data')
for file_name in file_names:
    url = f'https://raw.githubusercontent.com/thunlp/FewRel/master/data/{file_name}.json'

    response = requests.get(url)
    data = response.json()
    with open(f'./data/few_rel/{file_name}.json','w') as d:
        json.dump(data)


In [18]:
from source.ngram_utils import find_token_range

In [20]:
tokenizer('TJQ').tokens()

['[CLS]', 't', '##j', '##q', '[SEP]']

In [25]:
find_token_range(tokenizer.tokenize(txt),tokenizer.tokenize('tanjung pandan'))

[[16, 21]]

In [33]:
e1_id = tokenizer.convert_tokens_to_ids('[ENTITY1]')
e1_id

28996

In [34]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(29000, 768)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
   

In [ ]:
- Enter entity token around the sequence
- Replace with equal length entity token E1, E1, \E1
- Replace with single entity token

In [8]:
print(js_ls['P931'][0])

{'tokens': ['Merpati', 'flight', '106', 'departed', 'Jakarta', '(', 'CGK', ')', 'on', 'a', 'domestic', 'flight', 'to', 'Tanjung', 'Pandan', '(', 'TJQ', ')', '.'], 'h': ['tjq', 'Q1331049', [[16]]], 't': ['tanjung pandan', 'Q3056359', [[13, 14]]]}
